In [1]:
from pyscf import gto, scf, mp, adc, cc, lib, ao2mo
import matplotlib.pyplot as plt
import numpy as np
import dyson

In [2]:
# Build an MP2 self-energy and compress it - obtains a truncated MP2 energy,
# and gives access to Dyson-like ADC(2) excitations in place of the non-Dyson
# standard implementation.

nmom_lanczos = 4
nmom_projection = 3

In [3]:
# Get the PySCF methods:

mol = gto.M(atom='O 0 0 0; H 0 0 1; H 0 1 0', basis='cc-pvdz', verbose=0)
rhf = scf.RHF(mol).run(conv_tol=1e-12)
mp2 = mp.MP2(rhf).run()
adc2 = adc.ADC(rhf)
nmo = rhf.mo_occ.size

print('MP2 energy:', mp2.e_corr)
print('non-Dyson ADC(2) IP:', adc2.run().ip_adc()[0])
print('non-Dyson ADC(2) EA:', adc2.run().ea_adc()[0])

MP2 energy: -0.20905684800797256
non-Dyson ADC(2) IP: 0.3984057507817479
non-Dyson ADC(2) EA: 0.15307443795091602


In [4]:
# Build the moments of the occupied and virtual self-energies:

ei = rhf.mo_energy[rhf.mo_occ > 0]
ea = rhf.mo_energy[rhf.mo_occ == 0]
eija = lib.direct_sum('i+j-a->ija', ei, ei, ea)
eabi = lib.direct_sum('a+b-i->abi', ea, ea, ei)

cx = rhf.mo_coeff
ci = rhf.mo_coeff[:, rhf.mo_occ > 0]
ca = rhf.mo_coeff[:, rhf.mo_occ == 0]
xija = ao2mo.incore.general(rhf._eri, (cx,ci,ci,ca), compact=False).reshape([x.shape[1] for x in (cx,ci,ci,ca)])
xabi = ao2mo.incore.general(rhf._eri, (cx,ca,ca,ci), compact=False).reshape([x.shape[1] for x in (cx,ca,ca,ci)])

t_occ = np.einsum('xija,yija,nija->nxy', xija, 2*xija-xija.swapaxes(1,2), eija[None]**np.arange(2*nmom_lanczos+2)[:,None,None,None])
t_vir = np.einsum('xabi,yabi,nabi->nxy', xabi, 2*xabi-xabi.swapaxes(1,2), eabi[None]**np.arange(2*nmom_lanczos+2)[:,None,None,None])

In [9]:
# Perform the truncation and check consistency in the moments - separate occupied 
# and virtual moments should only match if nmom_projection is None.

e_red, v_red = dyson.kernel_se(t_occ, t_vir, nmom_lanczos, nmom_projection, phys=np.diag(rhf.mo_energy))

e_occ, v_occ = e_red[e_red < 0], v_red[:, e_red < 0]
e_vir, v_vir = e_red[e_red >= 0], v_red[:, e_red >= 0]

t_red = np.einsum('xk,yk,nk->nxy', v_red, v_red, e_red[None]**np.arange(2*nmom_lanczos+2)[:,None])
t_occ_red = np.einsum('xk,yk,nk->nxy', v_occ, v_occ, e_occ[None]**np.arange(2*nmom_lanczos+2)[:,None])
t_vir_red = np.einsum('xk,yk,nk->nxy', v_vir, v_vir, e_vir[None]**np.arange(2*nmom_lanczos+2)[:,None])

all_matches = [np.allclose(x/np.max(x), y/np.max(x)) for x,y in zip(t_occ+t_vir, t_red)][:2*min(nmom_lanczos, nmom_projection)+2]
occ_matches = [np.allclose(x/np.max(x), y/np.max(x)) for x,y in zip(t_occ, t_occ_red)][:2*min(nmom_lanczos, nmom_projection)+2]
vir_matches = [np.allclose(x/np.max(x), y/np.max(x)) for x,y in zip(t_vir, t_vir_red)][:2*min(nmom_lanczos, nmom_projection)+2]

print('Central moments match:', all(all_matches))
print('Occupied moments match:', all(occ_matches))
print('Virtual moments match:', all(vir_matches))

[True, True, True, True, True, True, True, True]
Central moments match: True
Occupied moments match: False
Virtual moments match: False


In [6]:
# Get the truncated MP2 energy (will approach MP2 in the limit of infinite moments):

v = v_red[rhf.mo_occ > 0][:, e_red >= 0]
d = ei[:, None] - e_red[e_red >= 0][None]
e_mp2 = np.einsum('xk,xk,xk->', v, v.conj(), 1.0/d)

print('Compressed MP2 energy:', e_mp2)

Compressed MP2 energy: -0.20879318952488593


In [7]:
# Get the truncated Dyson ADC(2) IP and EA (will approach Dyson ADC(2) in the limit 
# of infinite moments):

w, c = np.linalg.eigh(dyson.linalg.build_spectral_matrix(np.diag(rhf.mo_energy), e_red, v_red))

print('Compressed Dyson ADC(2) IP:', -np.max(w[w < 0]))
print('Compressed Dyson ADC(2) EA:', np.min(w[w >= 0]))

Compressed Dyson ADC(2) IP: 0.40236183053860997
Compressed Dyson ADC(2) EA: 0.1537265459917055
